In [1]:
import os
if not os.path.exists('exchange_rate_analysis'):
    !git clone https://github.com/matthewhirokawa/exchange_rate_analysis.git
%cd exchange_rate_analysis

"\nimport os\nif not os.path.exists('exchange_rate_analysis'):\n    !git clone https://github.com/matthewhirokawa/exchange_rate_analysis.git\n%cd exchange_rate_analysis\n"

In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
monthly_sentiment_nltk = pd.read_csv('data/usa_china_extended_monthly_sentiment_nltk.csv')
#monthly_sentiment_nltk['Month'] = pd.to_datetime(monthly_sentiment_nltk['Month']).dt.to_period('M')

monthly_sentiment_textblob = pd.read_csv('data/usa_china_extended_monthly_sentiment_textblob.csv')
#monthly_sentiment_textblob['Month'] = pd.to_datetime(monthly_sentiment_textblob['Month']).dt.to_period('M')
monthly_sentiment_textblob.head()

,Month,Relation,Sentiment
0,2021-01,USA-China,0.023501
1,2021-02,USA-China,0.020157
2,2021-03,USA-China,0.010724
3,2021-04,USA-China,0.014211
4,2021-05,USA-China,0.009114


In [4]:
usd_cny_exchange_rate = pd.read_csv('data/usd_cny_monthly_exchange_rate_2021.csv')
#usd_cny_exchange_rate['Month'] = pd.to_datetime(usd_cny_exchange_rate['Month']).dt.to_period('M')
usd_cny_exchange_rate.head()

,DATE,EXCHUS,Month
0,2021-01-01,6.4672,2021-01
1,2021-02-01,6.4601,2021-02
2,2021-03-01,6.5109,2021-03
3,2021-04-01,6.5186,2021-04
4,2021-05-01,6.4321,2021-05


In [5]:
#combine dataframes, drop date keep month
combined_df_textblob = pd.merge(monthly_sentiment_textblob, usd_cny_exchange_rate, on = 'Month', how = 'inner')
combined_df_textblob = combined_df_textblob.drop(columns=['DATE'])
#combined_df_textblob.to_csv("combined_df_textblob.csv", index = False)
combined_df_textblob.head()

,Month,Relation,Sentiment,EXCHUS
0,2021-01,USA-China,0.023501,6.4672
1,2021-02,USA-China,0.020157,6.4601
2,2021-03,USA-China,0.010724,6.5109
3,2021-04,USA-China,0.014211,6.5186
4,2021-05,USA-China,0.009114,6.4321


In [6]:
#create linear regression
df = pd.read_csv('data/combined_df_textblob.csv', index_col = False)
df = df.dropna()

y = df['Log_EXCHUS_Lag']
x = df[['Sentiment_Change']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

model = LinearRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Test MSE:", mse)
print("Test R-squared:", r2)
print("Model coefficient:", model.coef_[0])
print("Model intercept:", model.intercept_)

df.head()

Test MSE: 0.00013004019210459397
Test R-squared: -0.045083615802483656
Model coefficient: -0.06998869326199071
Model intercept: 0.0023479349016223934


,Month,Relation,Sentiment,EXCHUS,Log_EXCHUS,Log_Sentiment,Sentiment_Change,Log_EXCHUS_Lag
1,2021-02,USA-China,0.020157,6.4601,-0.001098,-0.153487936,-0.003344,0.007833
2,2021-03,USA-China,0.010724,6.5109,0.007833,-0.63106134,-0.009433,0.001182
3,2021-04,USA-China,0.014211,6.5186,0.001182,0.281525595,0.003487,-0.013359
4,2021-05,USA-China,0.009114,6.4321,-0.013359,-0.444211891,-0.005097,-0.001104
5,2021-06,USA-China,0.008897,6.4250,-0.001104,-0.024057627,-0.000217,0.007953


In [7]:
#model summary of sentiment and log exchange rate, lagged by 1 period
x = df[['Sentiment_Change']]

# Add constant (intercept) term
x = sm.add_constant(x)

# Dependent variable
y = df['Log_EXCHUS_Lag']

# Fit the OLS model
model = sm.OLS(y, x).fit()

# Print the summary
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:         Log_EXCHUS_Lag   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.019
Method:                 Least Squares   F-statistic:                   0.03007
Date:                Sat, 02 Aug 2025   Prob (F-statistic):              0.863
Time:                        14:12:48   Log-Likelihood:                 155.68
No. Observations:                  52   AIC:                            -307.4
Df Residuals:                      50   BIC:                            -303.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0020      0.002  

In [8]:
#read in extra data from excel
df = pd.read_csv('data/multi_regression.csv', index_col = False)
df = df.dropna()
df.head()

,Month,Relation,Sentiment,EXCHUS,US_tariff_rate,China_tariff_rate,US_exports_millions,China_exports_millions,Log_EXCHUS_Change,Log_Sentiment_Change,Sentiment_Change,Log_EXCHUS_Change_Lag,US_log_tariff_rate_change_lag,China_log_tariff_rate_change_lag,US_log_exports_change_lag,China_log_exports_change_lag
1,2021-02,USA-China,0.020157,6.4601,19.3,21.1,"9,265.10","33,816.80",-0.001098,-0.153491351,-0.003344,0.007833,0.0,0.0,0.324128,0.166270
2,2021-03,USA-China,0.010724,6.5109,19.3,21.1,"12,812.00","39,934.00",0.007833,-0.631067402,-0.009433,0.001182,0.0,0.0,-0.071209,-0.066330
3,2021-04,USA-China,0.014211,6.5186,19.3,21.1,"11,931.40","37,371.10",0.001182,0.281532092,0.003487,-0.013359,0.0,0.0,0.039119,0.029410
4,2021-05,USA-China,0.009114,6.4321,19.3,21.1,"12,407.40","38,486.50",-0.013359,-0.44420462,-0.005097,-0.001104,0.0,0.0,-0.021573,0.033039
5,2021-06,USA-China,0.008897,6.4250,19.3,21.1,"12,142.60","39,779.30",-0.001104,-0.024097552,-0.000217,0.007953,0.0,0.0,-0.032246,0.012779


In [9]:
#run multiregression with flipped x and y to check relationships
y = df['Sentiment_Change']
x = df[['Log_EXCHUS_Change_Lag', 'US_log_tariff_rate_change_lag', 'China_log_tariff_rate_change_lag', 'US_log_exports_change_lag', 'China_log_exports_change_lag']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

model = LinearRegression()
model.fit(x_train, y_train)

# Add constant (intercept) term
x = sm.add_constant(x)
# Fit the OLS model
model = sm.OLS(y, x).fit()

# Print the summary
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:       Sentiment_Change   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.8830
Date:                Sat, 02 Aug 2025   Prob (F-statistic):              0.500
Time:                        14:12:48   Log-Likelihood:                 162.58
No. Observations:                  51   AIC:                            -313.2
Df Residuals:                      45   BIC:                            -301.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   

In [24]:
#run regression of only sentiment change and US export change, export change is dependent variable
x = df[['Sentiment_Change']]
y = df['US_log_exports_change_lag']

# Add constant (intercept) term
x = sm.add_constant(x)
# Fit the OLS model
model = sm.OLS(y, x).fit()

# Print the summary
print(model.summary())

                                OLS Regression Results                               
Dep. Variable:     US_log_exports_change_lag   R-squared:                       0.079
Model:                                   OLS   Adj. R-squared:                  0.060
Method:                        Least Squares   F-statistic:                     4.187
Date:                       Sat, 02 Aug 2025   Prob (F-statistic):             0.0461
Time:                               14:14:10   Log-Likelihood:                 29.141
No. Observations:                         51   AIC:                            -54.28
Df Residuals:                             49   BIC:                            -50.42
Df Model:                                  1                                         
Covariance Type:                   nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------